In [1]:
import pandas as pd
import numpy as np
import json
from collections import Counter
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import re, string, unicodedata
import nltk
from nltk import word_tokenize, sent_tokenize, FreqDist
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer
nltk.download
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.tokenize import TweetTokenizer

[nltk_data] Downloading package wordnet to C:\Users\Ashima
[nltk_data]     Singhal\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Ashima
[nltk_data]     Singhal\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [32]:
df_final = pd.read_csv(filepathname)

In [64]:
df_final['hashtag'] = df_final['Text'].apply(lambda x: re.findall(r"#(\w+)", str(x)))

In [65]:
def preprocess_data(data):
 #Removes Numbers
    data = data.astype(str).str.replace('\d+', '')
    lower_text = data.str.lower()
    lemmatizer = nltk.stem.WordNetLemmatizer()
    w_tokenizer =  TweetTokenizer()
 
    def lemmatize_text(text):
        return [(lemmatizer.lemmatize(w)) for w \
                       in w_tokenizer.tokenize((text))]
    def remove_punctuation(words):
        new_words = []
        for word in words:
            new_word = re.sub(r'[^\w\s]', '', (word))
            if new_word != '':
                new_words.append(new_word)
        return new_words
    words = lower_text.apply(lemmatize_text)
    words = words.apply(remove_punctuation)
    return pd.DataFrame(words)


In [66]:
pre_tweets = preprocess_data(df_final['Text'])
df_final['text'] = pre_tweets
stop_words = set(stopwords.words('english'))
df_final['text'] = df_final['text'].apply(lambda x: [item for item in \
                                    x if item not in stop_words])

In [67]:
df_final

,Unnamed: 0,UserScreenName,UserName,Timestamp,Text,Embedded_text,Emojis,Comments,Likes,Retweets,Image link,Tweet URL,hashtag,text
0,0,Dr. Danielle Lawson,@DanielleFLawson,2021-04-01T23:54:34.000Z,"“How do we cover from COVID, environmental dis...",NaN,NaN,1,1,6,[],https://twitter.com/DanielleFLawson/status/137...,[],"[cover, covid, environmental, disaster, puttin..."
1,1,Park Wolfpack Wrestling,@WolfpackPark,2021-04-01T23:58:34.000Z,JC has been awarded our - hopefully - only eve...,NaN,NaN,NaN,NaN,4,['https://pbs.twimg.com/media/Ex7UswQWYAAS6gN?...,https://twitter.com/WolfpackPark/status/137777...,[],"[jc, ha, awarded, hopefully, ever, covid, awar..."
2,2,The Citizen Pundit,@1citizenpundit,2021-04-01T23:53:56.000Z,Employers can't require Covid-19 vaccination u...,Employers can't require Covid-19 vaccination u...,NaN,1,6,8,['https://pbs.twimg.com/card_img/1386726853117...,https://twitter.com/1citizenpundit/status/1377...,[],"[employer, cant, require, covid, vaccination, ..."
3,3,#SurvivedAndPunished,@COLBYLENZ,2021-04-01T23:54:58.000Z,People with #COVID19 at CCWF women's prisons a...,NaN,NaN,1,20,36,[],https://twitter.com/COLBYLENZ/status/137777150...,[COVID19],"[people, covid, ccwf, womens, prison, currentl..."
4,4,justjessylynn,@jessylynn40,2021-04-01T23:55:38.000Z,This is why we can’t have nice things. 552k de...,Quote Tweet\nKelliX84 (Pantyhose Priestess)\n@...,NaN,NaN,2,3,['https://pbs.twimg.com/profile_images/1375863...,https://twitter.com/jessylynn40/status/1377771...,[COVID19],"[nice, thing, k, dead, american, still, fighti..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2654,2654,Marcella Bernardo,@Bernardo1130,2021-03-29T23:53:35.000Z,Key information from #BC's #COVID19 update:\n@...,Quote Tweet\nNEWS 1130\n@NEWS1130\n · Mar 30\n...,NaN,8,9,19,['https://pbs.twimg.com/profile_images/1158815...,https://twitter.com/Bernardo1130/status/137668...,"[BC, COVID19, bcpoli]","[key, information, bcs, covid, update, whistle..."
2655,2655,Bernews,@bernewsdotcom,2021-03-29T23:59:02.000Z,The Ministry received 3038 test results and 10...,"Covid: 108 New Cases, Now 396 Active - Bernews...",NaN,1,4,3,['https://pbs.twimg.com/card_img/1387707967131...,https://twitter.com/bernewsdotcom/status/13766...,[],"[ministry, received, test, result, positive, c..."
2656,2656,Adrian Dix,@adriandix,2021-03-29T23:56:13.000Z,Here is my joint statement w/ Dr. Bonnie Henry...,"Joint statement on B.C.’s COVID-19 response, l...",NaN,10,8,7,['https://pbs.twimg.com/card_img/1386847647302...,https://twitter.com/adriandix/status/137668465...,"[COVID19, BC]","[joint, statement, w, dr, bonnie, henry, today..."
2657,2657,Santa Fe Reporter,@SantaFeReporter,2021-03-29T23:55:47.000Z,The state is making the process for vaccinatio...,NM Eliminates Vaccine Codes for Seniors\nHealt...,NaN,1,1,7,['https://pbs.twimg.com/card_img/1387229373728...,https://twitter.com/SantaFeReporter/status/137...,[COVID19],"[state, making, process, vaccination, easier, ..."


In [71]:
from ekphrasis.classes.segmenter import Segmenter
# segmenter using the word statistics from Twitter
seg_tw = Segmenter(corpus="twitter")
a = []
for i in range(len(df_final)):
    if df_final['hashtag'][i] != a:
        listToStr1 = ' '.join([str(elem) for elem in \
                                       df_final['hashtag'][i]])
        df_final.loc[i,'Segmented#'] = seg_tw.segment(listToStr1)

Reading twitter - 1grams ...
Reading twitter - 2grams ...


ValueError: Lengths must match to compare

In [50]:
df_final.to_csv(r'C:\Users\Ashima Singhal\Desktop\scweet\output\df_final.csv')

In [55]:
for tweet in df_final['Text']:
    tweet = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",str(tweet)).split())
    print(tweet)

How do we cover from COVID environmental disasters and more By putting JUSTICE in the CENTER Dr Robert Bullard
JC has been awarded our hopefully only ever Covid Award Through quarantines that took away a lot of his season he stuck with us working hard despite the set backs
Employers can t require Covid 19 vaccination under an EUA STAT
People with COVID19 at CCWF women s prisons are currently being held in the gym crowded in a corner with bunks crammed next to each other their sleep interrupted by guards watching TV loudly all night Most of those who are sick were already fully vaccinated
This is why we can t have nice things 552k dead Americans and still fighting COVID19
Cnn The AZ vacine is 90 pc ineffective against the south African variant of covid 19 We are totally tory fooked
I just got my first dose of the covid vaccine legendary
A viral variant has become a dominant source of COVID 19 in Minnesota causing as many as 65 of new infections and driving the state s case and hospitali

You had once locked her up for NOTHING And look at her today she came back doing what we expected you two to deliver instead you both chose to spew venom Covid You two are topping the chart of person of hatred in India congratulations
How To Interpret Data On COVID Vaccinations Case Numbers And Death Counts
Urgently need plasma from a recently recovered covid patient Name Vijay Bhat Blood Group A Location Max Hospital Patparganj Delhi Pamila Dhar 9717499888 Vijay Bhat 9811594333 If anyone can help Please do
Once again temporary solutions will be find out to tackle the permanent problems of poor healthcare infrastructure India Covid ICU beds oxygen decreasing sharply in Delhi says Arvind Kejriwal as city reports 24 000 Covid cases via
If you have recently recovered from COVID 19 you may be able to help patients currently fighting the infection by donating your plasma
People can t access COVID19 tests If people can t access tests they can t access treatment hospitals since they don t hav

Delhi Police Commissioner instructs officials to help public amid COVID crisis COVID19 DelhiPolice
As the second wave of COVID 19 hits the country badly your safety is in your own hands Not only wearing a mask is a necessity but you should also know the right method to wear it Let s have a look at that with YouthVeerangnayen StayHomeStaySafe
Some hospital have already run out of ICU beds in Malaysia For both COVID and Non COVID Stay home Stay safe
Arundhati Roy on India s Covid catastrophe We are witnessing a crime against humanity
Got my second covid shot So far so good We will see how I feel later
The family of a Toronto area nurse struggling with COVID 19 is speaking out tonight after they say she hasn t received pay while in hospital even though they believe she became infected with the virus at work shares their story
COVID 19 vaccines are FREE and the best thing we can do to protect ourselves and others Appts available at Hillsboro Stadium
Great disparities in COVID 19 vaccinatio

Breaking By end of April India is expected to get the first batch of Russian COVID19Vaccine E
Here s your dose of motivation if at all you ever needed one GetVaccinated RegisterforVaccination GetTheShot COVID19Vaccine VaccinesSaveLives CoronavirusVaccine coronavirus vaccinated vaccination IndiaFightsBack IndiaFightsCorona
reduces the price of Covishield to the states from Rs 400 to Rs 300 per dose effective immediately
COVID19Vaccines Adar Poonawalla says We have taken a philanthropic decision to reduce the price of Covishield for states by 25 percent to Rs 300 dose effective immediately Doing so in the interest of saving states funds
Breaking CEO tweets that the price of Covishield vaccine to the states has been slashed from Rs 400 to Rs 300 per dose with immediate effect
Y category security for a man showing philantropic gesture What times we are living in COVID19 Covishield CovidIndia
No 2nd dose covishield available for Senior citizens at Faridabad hospitals hospital
Negative RT PC

First Covishield vaccine or Don t die while trying to live FaceShieldsAndMasks
This is a perfect alert However my father got registered but not my mother but they both were administered Covishield so keep the slip provided by he hospital all the time with you hospital staff experiencing issues registering the details of the people who took the jab
Are we pricing vaccines or selling stuff at Parchun Ki Dukan 400 ki Nahin 300 Ki Le Ja Sensible way is Cost Transparently put cost per doze including R D etc in public domain charge a modest mark up Same for
Co WIN Registration Step by Step Guide to Register for 1st jab of COVID19Vaccine using CoWin Portal and AarogyaSetu App COVIDEmergency CowinDown HelpingHand cowinregistration CowinPortal
Create an alternate method of registration for vaccination for people in Hilly areas Uttrakhand High court UHC says there is internet connectivity issue in the hilly regions of the State COVID19Vaccine
why not price be reduced to half for private hospital

Looking forward to being on tonight with Michaelson and to talk about many compelling issues surrounding this past year during Covid Tune in 7pm
Chancellor Tours SUNY Plattsburgh COVID Test Site
Sad but needs to be seen COVID19 CovidVaccine
A strong collective effort from local leaders south of the Fraser have dramatically reduced Covid 19 infection rates within the South Asian community Fantastic leadership and teamwork in action cdnpoli bcpoli SurreyBC
Well thank the gods Im so tired of covid and men Sup ladies
The Makati City government has procured 42 biomedical refrigerators and laboratory freezers in preparation for the arrival of the COVID 19 vaccines
Boeing executives say requiring COVID 19 tests before U S flights could pose economic risks
BREAKING Members of the Texas State Board of Education SBOE are calling on the state s COVID 19 Expert Vaccine Allocation Panel to include public school teachers and staff in Phase 1B of COVID 19 vaccinations
Due to positive movement on the 

FB memory Lots of adults with their chins on their chests about lockdown and Covid19 and vaccines and Just get on with it The kids have had the worst possible time being away from their mates hobbies and sports Getting intolerant of the whinging It s not the blitz
The Johnson Johnson vaccine is now headed to NYC to the benefit of thousands of our residents Another lifesaving vaccine is good news for New York City Leading scientists agree the Johnson Johnson vaccine is safe and will protect against COVID19
FDA Authorizes Johnson Johnson s Single Dose COVID 19 Vaccine
With a 3rd safe and effective COVID 19 vaccine now authorized by the FDA one which takes only a single shot to protect those who receive it we will keep distributing vaccines quickly equitably until all Oregonians who want a vaccine can have one
Second consecutive day with a record number of US COVID vaccinations 2 4 million today At this rate you ll be able to safely attend Comic Con July 22 25 as Lady Hellboy
Yeah covid 1

Parents How long are we going to let gaslight children Adults are more of a threat IF YOU HAVE SYMPTOMS remember there are many viruses similar to COVID remember flu Asymptomatic children are close to NO risk NoSymptomsNoSwab
U S COVID update Record 3 5 million vaccinations in one day New cases 77 339 Positivity rate 4 6 0 1 In hospital 40 325 183 In ICU 8 921 2 New deaths 1 265 Data HHS BNO JHU Vaccinations At least 1 dose 91 7M 2 1M Fully vaccinated 50 1M 1 4M
A French monastery in Burgundy launched an emergency online sale to get rid of thousands of its prized artisanal cheeses which are languishing in its cellars as Covid 19 keeps buyers away It seemed prayers were answered when orders came in quickly
During the first wave a staggering 108 residents died of COVID19 at Sainte Doroth e long term care home north of Montreal During the second wave only one was infected and none died What changed by via eldercare
Weird things about COVID world My birthday and the day I get my 2nd vaccin

In [62]:
#Frequency of words
fdist = FreqDist(df_final['Segmented#'])
#WordCloud
wc = WordCloud(width=800, height=400, max_words=50).generate_from_frequencies(fdist)
plt.figure(figsize=(12,10))
plt.imshow(wc, interpolation="bilinear")
plt.axis("off")
plt.show()

KeyError: 'Segmented#'